# Vector-valued Poisson equation 

strong form of the boundary value problem

$$
\begin{align*}
&\text{Find}~\textbf{u}(\textbf{x}): \Omega \to \mathbb{R}^d~\text{such that} \\
&\begin{cases}
\nabla^2\textbf{u} = \textbf{f} & \forall\textbf{x}\in\Omega \\
\textbf{u}=\textbf{u}_{\text{D}} & \forall \textbf{x}\in\partial\Omega_{\text{D}} \\
\textbf{n}\cdot\nabla{\textbf{u}} = \textbf{u}_{\text{N}} & \forall\textbf{x}\in\partial\Omega_{\text{N}}=\partial\Omega/\partial\Omega_{\text{D}}
\end{cases}~.
\end{align*}
$$

weak form

$$
\begin{align*}
&\text{Find}~\textbf{u}\in V~\text{such that} \\
&F(\textbf{u}, \textbf{v})=-\int_\Omega\text{d}\Omega~\nabla\textbf{v}\cdot\nabla\textbf{u} + \textbf{v}\cdot\textbf{f} + \int_{\partial\Omega}\text{d}\Gamma~\textbf{v}\cdot(\textbf{n}\cdot\nabla\textbf{u})=0\quad\forall \textbf{v}\in V~.
\end{align*}
$$


## Vector-valued Poisson equation on a rectangle

$$
\mathbb{S}=
\begin{cases}
\Omega = [0, L_x] \times [0, L_y] \\
\textbf{f}(x,y)=(-y, x) \\
\textbf{u}_{\text{D}}(x,y=0)=(x, 1) \\
\textbf{u}_{\text{D}}(x,y=L_y)=(1, 2) \\
\textbf{u}_{\text{D}}(x=0,y)=(0, 0) \\
\textbf{u}_{\text{D}}(x=L_x,y)=(1, -1)
\end{cases}
$$

In [ ]:
from dolfinx.fem import FunctionSpace
from lucifex.mesh import rectangle_mesh, mesh_boundary
from lucifex.fem import Function, Constant
from lucifex.solver import bvp, BoundaryConditions
from lucifex.viz import plot_colormap, plot_line
from lucifex.utils import cross_section, get_component_fem_functions
from lucifex.pde.poisson import poisson


Lx = 2.0
Ly = 1.0
mesh = rectangle_mesh(Lx, Ly, 20, 10)
boundary = mesh_boundary(
    mesh, 
    {
        "left": lambda x: x[0],
        "right": lambda x: x[0] - Lx,
        "lower": lambda x: x[1],
        "upper": lambda x: x[1] - Ly,
    },
)
fs = FunctionSpace(mesh, ('P', 1, 2))
f = Function(fs, lambda x: (-x[1], x[0]), name='f')

bcs = BoundaryConditions(
    ('dirichlet', boundary['left'], (0.0, 0.0)),
    ('dirichlet', boundary['right'], Constant(mesh, (1.0, -1.0))),
    ('dirichlet', boundary['lower'], (lambda x: x[0], 1.0), 1),
    ('dirichlet', boundary['upper'], 1.0, 0),
    ('dirichlet', boundary['upper'], 2.0, 1),
)

u = Function(fs, name='u')
u_solver = bvp(poisson, bcs)(u, f)
u_solver.solve()

ux, uy = get_component_fem_functions(('P', 1), u, names=('ux', 'uy'))

In [ ]:
x_axis, ux_x, y_value = cross_section(ux, 'y', 0.5)
plot_line((x_axis, ux_x), x_label='$x$', y_label=f'{u.name}_x(y={y_value:.2f})')

y_axis, ux_y, x_value = cross_section(ux, 'x', 0.5)
plot_line((y_axis, ux_y), x_label='$y$', y_label=f'{u.name}_x(x={y_value:.2f})')

In [ ]:
x_axis, uy_x, y_value = cross_section(uy, 'y', 0.5)
plot_line((x_axis, uy_x), x_label='$x$', y_label=f'{u.name}_y(y={y_value:.2f})')

y_axis, uy_y, x_value = cross_section(uy, 'x', 0.5)
plot_line((y_axis, uy_x), x_label='$y$', y_label=f'{u.name}_y(x={y_value:.2f})')